In [18]:
using Distributed#, Plotly, DataFrames, Plots, PyPlot
#plotly()
# addprocs(2)

#@everywhere using StatsBase, Distributions
#@everywhere include("episiming.jl");

In [19]:
julia_f = "../Julia/Imperial College/SEIR/episiming.jl"
include(julia_f)

geraIdade! (generic function with 1 method)

In [20]:
function powerDecay(a::Number, b::Number, x)
    y = rowWiseNorm(x)
    return 1.0 ./ (1.0 .+ (y ./ a) .^ b)
end

fKernel(x) = powerDecay(1., 1.5, x)

fKernel (generic function with 1 method)

In [21]:
censoResidencial = [.21, .26, .20, .17, .08, .04, .02, 0.02];

In [22]:
geradorCenso(x) = geraResidencias(x, censoResidencial)
geradorDensidade(k) = (x) -> geraResidencias(x, k)

geradorDensidade (generic function with 1 method)

# Rio de Janeiro

In [35]:
##file = open("landScan.bin", "r")
#data = reinterpret(Int64, read(file))
#close(file)
dadosRJ = np.load("landscan_rio.npy")
#dadosRJ = reshape(data, (36, 76));
dadosRJ[dadosRJ .< 0] .= 0;

#PyPlot.imshow(log10.(dadosRJ))
#PyPlot.colorbar();

In [36]:
dadosAjustados = floor.(Int, dadosRJ ./ 1);

In [26]:
αᵣ = 0.8
βᵢ = 0.30
βₐ = 0.05

geradorθᵢ() = (self::Particula, x) -> βᵢ / self.n ^ αᵣ
geradorθₐ() = (self::Particula, x) -> βₐ / self.n ^ αᵣ

geradorθₐ (generic function with 1 method)

In [37]:
@time populacao = geraRedeResidencial(dadosAjustados, geradorCenso, geradorθᵢ, geradorθₐ, 100, fKernel);

 81.876124 seconds (139.51 M allocations: 84.965 GiB, 9.79% gc time)


In [28]:
using PyCall

In [29]:
dadosRJ

UndefVarError: UndefVarError: dadosRJ not defined

In [38]:
populacao.posicoes

6857853×2 Array{Float64,2}:
  0.382382  18.0546
  0.382382  18.0546
  0.382382  18.0546
  0.753743  18.9369
  0.10167   19.2854
  0.10167   19.2854
  0.10167   19.2854
  0.10167   19.2854
  0.228849  19.8886
  0.228849  19.8886
  0.228849  19.8886
  0.602456  20.2282
  0.826496  20.579
  ⋮         
 75.6372    25.5993
 75.6372    25.5993
 75.6372    25.5993
 75.6372    25.5993
 75.6372    25.5993
 75.6372    25.5993
 75.6372    25.5993
 75.5268    25.3334
 75.5268    25.3334
 75.5268    25.3334
 75.8259    25.508
 75.8259    25.508

In [31]:
py"""
import numpy as np
import random 

def corrige_subnotificacao(mtrx_infectados):
    return mtrx_infectados*8

def corrige_mtrx(mtrx):
        new_mtrx = np.zeros(np.shape(mtrx))
        for i in range(39):
            new_mtrx[np.abs(i-38)] = mtrx[i]
        return new_mtrx

def reducao_mtrx(tx_reducao, mtrx):
    if tx_reducao == 1:
        mtrx_reduzida = mtrx
    else:
        qt_total = np.rint(np.sum(mtrx)/tx_reducao)
        rng_blocos = np.arange(np.prod(np.shape(mtrx)))
        pesos = (mtrx/np.sum(mtrx)).flatten()
        escolha = np.array(random.choices(rng_blocos, pesos, k = int(qt_total)))

        rows = np.floor(escolha/83)
        cols = np.mod(escolha,83)
        mtrx_reduzida = np.zeros((39,83))
        for i,j in zip(rows,cols):
            mtrx_reduzida[int(i)][j] += 1
    return mtrx_reduzida

def distribuicao_inicial_casos(tx_reducao, mtrx_casos, pop_posicoes):
    pos_indv_blocos = np.round(pop_posicoes)
    mtrx_casos_reduzida = reducao_mtrx(tx_reducao, mtrx_casos)

    casos = []
    rng = np.arange(len(pos_indv_blocos))
    for x in range(83):
        for y in range(39):
            checa_bloco_indv = pos_indv_blocos == [x,y]
            indv_nos_blocos =  np.logical_and(checa_bloco_indv[:,0], checa_bloco_indv[:,1])
            indices_no_bloco = rng[indv_nos_blocos]
            qt_bloco = mtrx_casos_reduzida[y,x]                  
            if (qt_bloco > 0) & (len(indices_no_bloco) > 0):
                indv_casos = np.random.choice(indices_no_bloco, int(qt_bloco))
                for e in indv_casos:
                    casos.append(e)
    return casos
"""

In [39]:
np.save("posicao_cenario_completo", populacao.posicoes)

In [33]:
dist_infectados = py"distribuicao_inicial_casos"(1, infectados*8, populacao.posicoes)

UndefVarError: UndefVarError: infectados not defined

In [9]:
using Pkg

In [11]:
Pkg.add("pyimport")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


    Fetching: [=================>                       ]  

    Fetching: [===============================

Fetching: [========================================>]  100.0 %

Pkg.Types.PkgError: The following package names could not be resolved:
 * pyimport (not found in project, manifest or registry)


In [12]:
sum(infectados*8)

UndefVarError: UndefVarError: infectados not defined

In [34]:
np = pyimport("numpy")

PyObject <module 'numpy' from '/home/gil/.local/lib/python3.8/site-packages/numpy/__init__.py'>

In [24]:
infectados = np.load("infectados.npy")

39×83 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [16]:
Pkg.add("NPZ")

  Resolving package versions...
  Installed Zlib_jll ─ v1.2.11+14
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
  [83775a58] ↑ Zlib_jll v1.2.11+13 ⇒ v1.2.11+14


In [9]:
mutable struct SEAIR <: Parametros
    fKernel
    γ::Number
    probFimIncubacao::Number
    probAssintomatico::Number
    θᵢ
    θₐ
    cargaViral
end

In [10]:
dist = calculaDistancia(populacao, fKernel);
#@eval @everywhere dist=$dist

In [28]:
# ρForma = 0.2 # shape factor of gamma distribution
# ρEscala = 5 # scale (mean value = scale * shape)
# ρPop = rand(Gamma(ρForma, ρEscala), nPessoas)

βᵤᵢ = 0.06
βᵤₐ = 0.006
γ = 0.1;

parametros = SEAIR(fKernel, γ, 1/3, 4/6, (t) -> βᵤᵢ ./ dist, (t) -> βᵤₐ ./ dist, (a, t) -> ones(length(a)));

In [42]:
function sendto(p::Int; args...)
    for (nm, val) in args
        @spawnat(p, eval(Main, Expr(:(=), nm, val)))
    end
end

function sendto(ps::Vector{Int}; args...)
    for p in ps
        sendto(p; args...)
    end
end

macro evolucaoParalela(populacao, tempos, parametros, nSim)
    sendto(workers(), fKernel=eval(parametros).fKernel)
    return :(pmap((i)->evolucaoMista($populacao, $tempos, $parametros), 1:$nSim))
end

@evolucaoParalela (macro with 1 method)

In [62]:
using CodeTracking, Revise

In [63]:
CodeTracking.code_expr(fKernel, [Array])

In [45]:
@time a = @evolucaoParalela(populacao, Array(1:2), parametros, 4)

RemoteException: On worker 3:
UndefVarError: parametros not defined
#108 at ./In[42]:15
#104 at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Distributed/src/process_messages.jl:294
run_work_thunk at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Distributed/src/process_messages.jl:79
macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Distributed/src/process_messages.jl:294 [inlined]
#103 at ./task.jl:358

In [27]:
using BenchmarkTools

In [29]:
@btime a = pmap((i)->evolucaoMista(populacao, Array(1:2), parametros), 1:4)

  1.071 s (803 allocations: 6.25 MiB)


4-element Array{Tuple{Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Int64,2}},1}:
 ([67873.0, 0.0], [0.0, 62.0], [0.0, 45598.0], [100.0, 22313.0], [0.0, 0.0], [-1 1 -1; -1 1 -1; … ; -1 1 -1; -1 1 -1])
 ([67873.0, 0.0], [0.0, 61.0], [0.0, 45143.0], [100.0, 22769.0], [0.0, 0.0], [-1 1 -1; -1 1 -1; … ; -1 1 -1; -1 1 -1])
 ([67873.0, 0.0], [0.0, 66.0], [0.0, 45271.0], [100.0, 22636.0], [0.0, 0.0], [-1 1 -1; -1 1 -1; … ; -1 1 -1; -1 1 -1])
 ([67873.0, 0.0], [0.0, 62.0], [0.0, 45368.0], [100.0, 22543.0], [0.0, 0.0], [-1 1 -1; -1 1 -1; … ; -1 1 -1; -1 1 -1])

In [30]:
@btime a = evolucaoMista(populacao, Array(1:2), parametros);

  510.104 ms (5920977 allocations: 284.25 MiB)


In [35]:
function evolucaoMista(
    populacao::Populacao, tempos::AbstractArray{T} where T <: Number, parametros::Parametros; timing=false)
    """
        Entrada:
            populacao: 
            tempos:
            θᵤ: função para encontrar taxa de transmissão global
            γ: parâmetro da exponencial de não recuperação
            fKernel: ?
    """
    nT = length(tempos)
    passos = tempos[2:end] - tempos[1:(end-1)]

    S = zeros(nT)
    E = zeros(nT)
    I = zeros(nT)
    A = zeros(nT)
    R = zeros(nT)
    transicoes = -1 .* ones(Int, populacao.n, 3)

    S[1] = populacao.n - populacao.I0
    I[1] = populacao.I0
    transicoes[populacao.estadoInicial .== 4, 1:2] .= 0

    estadoAtual = copy(populacao.estadoInicial)
    for (k, δ) in enumerate(passos)
        @time estadoAtual .= passoMisto(populacao, estadoAtual, k, δ, transicoes, parametros)
        
        S[k+1] = sum(estadoAtual .== 1)
        E[k+1] = sum(estadoAtual .== 2)
        A[k+1] = sum(estadoAtual .== 3)
        I[k+1] = sum(estadoAtual .== 4)
        R[k+1] = sum(estadoAtual .== 5)
    end

    return S,E,A,I,R,transicoes
end

evolucaoMista (generic function with 1 method)

In [13]:
calculaDistancia(populacao::Populacao, diaContagio, cargaViral, t, suscetiveis, infectados, fKernel)

UndefVarError: UndefVarError: diaContagio not defined

In [14]:
Plots.plot((mean(S, dims=1)[1, :]), label = "Suscetíveis")
Plots.plot!((mean(I, dims=1)[1, :]), label = "Infectados")
Plots.plot!((mean(R, dims=1)[1, :]), label = "Recuperados")

<!DOCTYPE html>
 
 
 Plots.jl

In [ ]:
GC.gc()